Funciones para calcular las coeficientes sigmas, esperanzas de
las distribuciones discretas que resultan de estas, etc... 

**Importando módulos**

In [ ]:
import math 
import numpy as np
import matplotlib.pyplot as plt
import pylab

In [5]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00


In [10]:
import base_fourier_V0 
import base_fourier_V1
import legendre_discreto as legendre
import minimos_cuadrados

**FUNCIONES**

In [11]:
#Función auxiliar
def esperanza(dominio, mediciones):
	"""
	Función que calcula la esperanza de la variable aleatoria cuyo dominio es la lista
	'dominio' y que toma los valores del vector 'mediciones'.
	Claro que se supone que las listas 'dominio' y 'mediciones' tienen la misma longitud,
	y que el i-ésimo valor de 'dominio' es mapeado al i-ésimo
	vector de 'mediciones'.'
	"""
	esp=0 #inicializamos la esperanza
	for i in range(len(dominio)):
		esp+=dominio[i]*mediciones[i]
	return esp


In [12]:
def calculando_sigmasYesp(N,k, fourier=base_fourier_V0):
	"""
	Función que calcula los coeficientes sigma
	del polinomio discreto de Legendre de dimensión N
	y grado k, junto con la esperanza de la distribución
	que ellos forman.
	"fourier" es un script en el que se ha definido una base de fourier
	(i.e. una base ortonormal de frecuencias).
	"""

	M=math.ceil(N/2)

	#inicializamos las bases de Fourier y Legendre discreta de dim N
	baseFourier=fourier.calculo_base(N)
	baseLegendre=legendre.calculo_base(N)

	#Llamamos al vector de Legendre de dim N y grado k
	vectorLegendre= baseLegendre[k]

	#Creando el vector de productos punto al cuadrado.
	prod_punto_cuadrado=[np.dot(vectorLegendre, baseFourier[mu])**2 for mu in range(N)] 

	#a partir de los productos punto al cuadrado almacenados en la lista de arriba calculamos los coeficientes sigma
	sigma=[prod_punto_cuadrado[0]] #inicializamos la lista con la primera entrada
	for l in range(1,M):
		sigma.append(prod_punto_cuadrado[2*l-1]+prod_punto_cuadrado[2*l])

	if N%2==1: #si N es impar, calculamos la esperanza y ya terminamos.
		dominio=[t for t in range(M)] #TODO: No sería mejor empezar desde uno? Para que la primera frecuencia no
									  #se elimine al calcular la esperanza...
		esp= esperanza(dominio, sigma)
		return (sigma, esp) 
	else: #en caso contrario, falta agregar una sigma y calcular la esperanza
		sigma.append(prod_punto_cuadrado[N-1])
		dominio=[t for t in range(M+1)] 
		esp= esperanza(dominio, sigma)
		return (sigma, esp) #para calcular esp necesito a sigma, por lo que no creo poder separar estos outputs

In [13]:
def graficando_sigmasYesp(N,k, fourier=base_fourier_V0):
	"""
	Función que grafica los coeficientes sigma
	del polinomio discreto de Legendre de dimensión N
	y grado k, junto con la esperanza de la distribución
	que ellos forman.
	"""
	sigma, esp = calculando_sigmasYesp(N,k, fourier) #calculamos los datos
	dominio=[t for t in range(len(sigma))]  #calculamos len(sigma) para no tener que hacer un if-else con la paridad.

	#graficando las sigmas
	plt.scatter(dominio, sigma, s=100, color="mediumpurple", marker="*")

	#graficando la esperanza (un solo punto).
	plt.scatter(esp, 0, s=100, color="darkgoldenrod", marker="^", label='Esperanza: '+str(esp.round(4))) #elegí la forma de una cuña (esperanza como punto de equilibrio)

	plt.xlabel("Frecuencia $\\omega$")
	plt.ylabel(r"$\sigma_{{\omega}}^{{ {0} }}( \mathcal{{ L }}^{{ {0} , {1} }} )$".format(str(N), str(k)) )
	plt.grid()
	plt.legend()
	plt.axhline(y=0, color='gray')	
	plt.axvline(x=0, color='gray')
	plt.title("La distribución $\\sigma_{{ {0} , {1} }}$ y su esperanza".format(str(N), str(k)) )
	

In [15]:
def graficando_esperanzas(N, fourier=base_fourier_V0):
	"""
	Función que calcula las esperanzas de los coeficientes sigma de cada uno de los N polinomios
	discretos de Legendre de grado N.
	
	NOTA: Observa cómo el primer punto siempre es cero. Esto se corresponde con el hecho
	de que todo polinomio discreto de Legendre de grado cero no tiene oscilaciones.
	TODO: fue bueno usar numpy arrays en lugar de arrays. Tienes que hacer esto para las demás funciones también.
	"""

	baseFourier=fourier.calculo_base(N)
	baseLegendre=legendre.calculo_base(N)

	dominio=np.array([t for t in range(N)])
	esperanzas=np.array([calculando_sigmasYesp(N,k, fourier)[1] for k in range(N)]) #iteramos en la variable de grado 'k'.

	plt.scatter(dominio, esperanzas, s=100, color="darkgoldenrod", marker="^")

	#Graficando la recta f(k)=k/2
	X=np.linspace(0, N, 100)
	plt.plot(X, X/2, color="black", linestyle='dashed', label="Gráfica de la recta $y=\\frac{1}{2}k$")
	b0, b1= minimos_cuadrados.coef_RMC(dominio, esperanzas)
	plt.plot(X, b1*X+b0, color="mediumblue", linestyle='dashed', label='Ajuste lineal de mínimos cuadrados')

	plt.xlabel("Grado $k$")
	plt.legend()
	plt.grid()
	plt.axhline(y=0, color='gray')	
	plt.axvline(x=0, color='gray')

	plt.title("Esperanzas de las distribuciones sigma de los pol. de Legendre de dimensión {0}".format(str(N)))

In [21]:
#graficando_sigmasYesp(11,7, base_fourier_V0) 

#base_legendre=legendre.calculo_base(11)
#print(np.dot(base_legendre[7], base_legendre[7]))